In [1]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 25.6 MB/s eta 0:00:00


In [7]:
import pandas as pd

# Read the original CSV file
df = pd.read_csv('train.csv')

# Select the first 200 rows
df_shrunk = df.head(1000)

# Save the shrunk dataframe to a new CSV file
df_shrunk.to_csv('train1000file.csv', index=False)

In [8]:
import pandas as pd
import requests
import numpy as np
import cv2
import easyocr
from concurrent.futures import ThreadPoolExecutor

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)  # Set gpu=True if you have GPU

# Function to extract text from an image URL
def extract_text_from_image(url):
    try:
        response = requests.get(url, timeout=10)  # Add a timeout for network requests
        image_arr = np.array(bytearray(response.content), dtype=np.uint8)
        img_1 = cv2.imdecode(image_arr, cv2.IMREAD_COLOR)
        text_result = reader.readtext(img_1, detail=0)
        return " ".join(text_result)
    except Exception as e:
        return ""

# Function to process a single row of the dataframe
def process_row(row):
    row['Extracted_Text'] = extract_text_from_image(row['image_link'])
    return row

# Function to process the entire CSV with multithreading
def process_csv_parallel(csv_path):
    df = pd.read_csv(csv_path)

    # Use ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor(max_workers=8) as executor:
        results = list(executor.map(process_row, [row for _, row in df.iterrows()]))

    # Convert results back to a DataFrame
    df = pd.DataFrame(results)

    # Overwrite the original CSV file with new columns
    df.to_csv(csv_path, index=False)
    return df

# Example usage
process_csv_parallel('train1000file.csv')


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

,image_link,group_id,entity_name,entity_value,Extracted_Text
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,PROPOS' NATUREJ INGREDIENT MENAGER MULTI-USAGE...
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,LLa eelE=_ 62 R RRIFICH LEBENSMITTELECHT Cwwv ...
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,COMPOSITION Serving Size: Tablet (0.709 g) Eac...
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,3 3 1 1 I I IW! # 5833 1 3 1 1 1 H 0 L 1 W # I...
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,Horbaach' HIGH StRE NGTH PSYLLIUM HUSK PLANTAG...
...,...,...,...,...,...
995,https://m.media-amazon.com/images/I/81uget9aH4...,639090,item_weight,470 gram,MK CHOCOLATE FLAVORED (e (00/66 9;t DL 0 MET U...
996,https://m.media-amazon.com/images/I/91QvLtiYaB...,365637,item_weight,454 gram,TRADITIONAL_ BuTTER COOKIES COPENHAGEN Oanisa...
997,https://m.media-amazon.com/images/I/81lazo6sxl...,601746,item_weight,4.0 ounce,2 #ripv: ~OUN K Tfpk ~uns 27 E8 8a E EAy 1 SO...
998,https://m.media-amazon.com/images/I/81fYO+HP5Q...,186035,voltage,0.5 volt,4 D % NON = LEE elte @nicldt Woactoortagiofy A...


In [4]:
df_shrunk = df.head(5)

df_shrunk.to_csv('train_5_file.csv', index=False)